In [1]:
import os
os.chdir("/home/mayank_khulbe_farmart_co/fmt/document-alignment-detection")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from document_alignment_correction.constants import label_mapping, train_params, test_params, pred_map, device
from document_alignment_correction.helper import HOG_Dataset, get_hog, rotateImage
from document_alignment_correction.model import DocumentAlignmentNet, train_cnn, evaluate_cnn

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

/home/mayank_khulbe_farmart_co/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_data = pd.read_csv('document_alignment_correction/data/train_data.csv')
train_data['path'] = "document_alignment_correction/" + train_data['path']
train_data.head()

,path,angles
0,document_alignment_correction/data/rotated_ima...,305
1,document_alignment_correction/data/rotated_ima...,234
2,document_alignment_correction/data/rotated_ima...,20
3,document_alignment_correction/data/rotated_ima...,166
4,document_alignment_correction/data/rotated_ima...,319


In [4]:
test_data = pd.read_csv('document_alignment_correction/data/test_data.csv')
test_data['path'] = "document_alignment_correction/" + test_data['path']
test_data.head()

,path,angles
0,document_alignment_correction/data/rotated_ima...,304
1,document_alignment_correction/data/rotated_ima...,95
2,document_alignment_correction/data/rotated_ima...,291
3,document_alignment_correction/data/rotated_ima...,21
4,document_alignment_correction/data/rotated_ima...,180


In [5]:
training_data= HOG_Dataset(train_data)
testing_data = HOG_Dataset(test_data)

training_loader = DataLoader(training_data, **train_params)
testing_loader = DataLoader(testing_data, **test_params)

model = DocumentAlignmentNet().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 40

#writer
# writer = SummaryWriter(f"logs/alignment_prediction_{fol_num}")

In [6]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [7]:
for epoch in range(num_epochs):
    
    #best model params
    best_model_state = None
    best_val_loss = 0.0
    epochs_without_improvement = 0
    
    #train and val loop
    train_loss, train_acc = train_cnn(model, criterion, optimizer, training_loader, epoch)
    print(f'Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')
    val_loss, val_acc = evaluate_cnn(model, criterion, testing_loader, epoch)
    print(f'Epoch [{epoch}/{num_epochs}], Test Loss: {val_loss:.4f}, Test Accuracy: {val_acc:.4f}')
    
    # scheduler.step()

    #early stopping
    if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= 4: #if model acc does not improve for 4 epochs
            print(f"Early stopping at epoch {epoch}.")
            break

100%|██████████| 25/25 [04:42<00:00, 11.29s/it]


Epoch [0/40], Train Loss: 1.4449, Train Accuracy: 0.3250


100%|██████████| 7/7 [01:45<00:00, 15.14s/it]


Epoch [0/40], Test Loss: 1.3792, Test Accuracy: 0.2545


100%|██████████| 25/25 [04:37<00:00, 11.11s/it]


...

Epoch [38/40], Test Loss: 0.0505, Test Accuracy: 0.9818


100%|██████████| 25/25 [04:42<00:00, 11.31s/it]


Epoch [39/40], Train Loss: 0.0163, Train Accuracy: 1.0000


100%|██████████| 7/7 [01:48<00:00, 15.43s/it]

Epoch [39/40], Test Loss: 0.0357, Test Accuracy: 1.0000


In [ ]:
nvidia-smi

## Prediction

In [ ]:
import torch
model = DocumentAlignmentNet()
model.load_state_dict(torch.load('cnn_model.pth'))
model

In [ ]:
pred_map

In [ ]:
# Open the document image
def get_prediction(path):
    hog_features = get_hog(path)
    pixels = training_data.transform(hog_features)
    # print()
    prediction = model.predict(pixels.unsqueeze(0))
    
    return pred_map.get(prediction.item())

In [ ]:
get_prediction("data/rotated_images/prediction/qkmg0065_1_180.png")